In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [3]:
arg_date = '2021-05-07'

In [4]:
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

In [5]:
arg_date_dt

datetime.date(2021, 5, 6)

In [6]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('deutsche-boerse-xetra-pds')

In [12]:
objects = [obj for obj in bucket.objects.all()
          if datetime.strptime(obj.key.split('/')[0], '%Y-%m-%d').date() >= arg_date_dt]

In [13]:
objects

[s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pd

In [14]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [64]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = df_all.append(df, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
df_all

In [ ]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [ ]:
df_all

In [ ]:
df_all.dropna(inplace=True)

In [ ]:
df_all.shape

## Get opening price per ISIN and day

In [ ]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [ ]:
df_all

In [ ]:
df_all[df_all['ISIN'] == 'AT0000A0E9W5']

## Get closing price per ISIN and day

In [ ]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [ ]:
df_all

In [ ]:
df_all[df_all['ISIN'] == 'AT0000A0E9W5']

## Aggregations

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'),
                                                             closing_price_eur=('closing_price', 'min'),
                                                             minimum_price_eur=('MinPrice', 'min'),
                                                             maximum_price_eur=('MaxPrice', 'max'),
                                                             daily_traded_volume=('TradedVolume', 'sum'))

In [ ]:
df_all

## Percent Change Prev Closing

In [ ]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [ ]:
df_all

In [ ]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [ ]:
df_all

In [ ]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [ ]:
df_all = df_all.round(decimals=2)

In [ ]:
df_all = df_all[df_all.Date >= arg_date]

In [53]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-03-15,9.49,9.41,9.30,9.60,2392,NaN
1,AT00000FACC2,2021-03-16,9.57,9.33,9.33,9.57,2280,-0.85
2,AT0000606306,2021-03-15,18.19,18.09,18.00,18.19,905,NaN
3,AT0000606306,2021-03-16,18.00,18.10,18.00,18.16,395,0.06
4,AT0000609607,2021-03-15,15.62,15.76,15.58,15.92,644,NaN
...,...,...,...,...,...,...,...,...
6040,XS2265369731,2021-03-15,10.19,10.15,10.15,10.22,0,NaN
6041,XS2265369731,2021-03-16,10.25,10.25,10.25,10.25,0,0.97
6042,XS2265370234,2021-03-15,19.79,20.05,19.79,20.05,50,NaN
6043,XS2284324667,2021-03-15,26.30,26.01,25.98,26.30,0,NaN


## Write to S3

In [ ]:
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [ ]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('xetra-12345')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)